In [ ]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from geometry import Geometry
from forcing import Forcing
from layer import LayerModel

from plotfunctions import prettyplot
import savetools as st

#np.set_printoptions(precision=2)

import warnings
warnings.filterwarnings('ignore')
np.seterr(all='ignore')

%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
dts = np.array([])
for n in np.arange(1,300):
    dtt = 8640/n
    if dtt==int(dtt):
        dts = np.append(dts,dtt)
dts = np.array(dts)
dts

In [ ]:
%%time

N = 1

#startyear = 2003
#endyear = 2008

geom = Geometry('CrossDots')
geom.coarsen(N=N)
geom = geom.create()

#forc = Forcing(geom).mitgcm1Db(startyear=startyear,endyear=endyear)
forc = Forcing(geom).tanh(ztcl=500,Tdeep=.5,z1=250)
#print(forc.Sz[-1].values,forc.Sz[-1200].values)

fig,ax = plt.subplots(1,3,sharey=True)
ax[0].plot(forc.Tz,forc.z)
ax[1].plot(forc.Sz,forc.z)
ax[2].plot(1028*(3.733e-5*(forc.Tz[-1]-forc.Tz)+7.843e-4*(forc.Sz-forc.Sz[-1])),forc.z)
ax[0].set_ylim([-1200,0])

In [ ]:

layer = LayerModel(forc)

layer.Ah = 25*N
layer.Kh = 25*N

layer.Cdtop = 1.1e-3
layer.minD = 2.8
layer.dt = 120

layer.saveday = 5
layer.restday = 100

ds = layer.compute(days=50)
#st.savefields(layer,filename=f'../../results/tuning/{0.5*N}_{layer.Cdtop*1e3:.1f}_{layer.minD}')

In [ ]:
from tools import *

fig,ax = plt.subplots(1,3,figsize=(20,10))
#ax[0].quiver(layer.x,layer.y,im(layer.u[1,:,:]),jm(layer.v[1,:,:]),zorder=10)
im = ax[0].pcolormesh(layer.x,layer.y,(im(layer.u[1,:,:])**2+jm(layer.v[1,:,:])**2)**.5,shading='auto',cmap='cmo.amp')
ax[0].set_title('Speed [m/s]')
plt.colorbar(im,ax=ax[0],orientation='horizontal')

im = ax[1].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.zb,np.nan),shading='auto',cmap='cmo.deep_r')
plt.colorbar(im,ax=ax[1],orientation='horizontal')
ax[1].set_title('basal topography [m]')
im = ax[2].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,3600*24*365.25*layer.melt,np.nan),shading='auto',cmap='inferno',norm=mpl.colors.LogNorm(vmin=1,vmax=100))#,vmin=-2e-6,vmax=2e-6)
plt.colorbar(im,ax=ax[2],orientation='horizontal')
ax[2].set_title('melt rate [m/s]')

In [ ]:
from tools import *

fig,ax = plt.subplots(1,3,figsize=(20,10))
#ax[0].quiver(layer.x,layer.y,im(layer.u[1,:,:]),jm(layer.v[1,:,:]),zorder=10)
im = ax[0].pcolormesh(layer.x,layer.y,(im(layer.u[1,:,:])**2+jm(layer.v[1,:,:])**2)**.5,shading='auto',cmap='cmo.amp')
plt.colorbar(im,ax=ax[0],orientation='horizontal')

im = ax[1].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.D[1,:,:],np.nan),vmin=0,vmax=1000,shading='auto',cmap='cmo.deep')
plt.colorbar(im,ax=ax[1],orientation='horizontal')

im = ax[2].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.zb-layer.D[1,:,:],np.nan),vmin=-800,vmax=0,shading='auto',cmap='gist_stern_r')
plt.colorbar(im,ax=ax[2],orientation='horizontal')

#for q in [0,1,2]:
#    ax[q].set_xlim([-1.51e6,-1.49e6])
#    ax[q].set_ylim([-640000,-620000])

In [ ]:

fig,ax = plt.subplots(1,3,figsize=(20,10))
im = ax[0].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.ent2-layer.detr,np.nan),shading='auto',cmap='cmo.balance',vmin=-1e-4,vmax=1e-4)
plt.colorbar(im,ax=ax[0],orientation='horizontal')

im = ax[1].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.entr,np.nan),shading='auto',cmap='cmo.balance',vmin=-1e-4,vmax=1e-4)
plt.colorbar(im,ax=ax[1],orientation='horizontal')

im = ax[2].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.melt,np.nan),shading='auto',cmap='cmo.balance',vmin=-1e-6,vmax=1e-6)
plt.colorbar(im,ax=ax[2],orientation='horizontal')

#for q in [0,1,2]:
#    ax[q].set_xlim([-1.51e6,-1.49e6])
#    ax[q].set_ylim([-640000,-620000])

In [ ]:

fig,ax = plt.subplots(1,3,figsize=(20,10))
im = ax[0].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.drho,np.nan),shading='auto',cmap='cmo.amp',vmin=0,vmax=5e-4)
plt.colorbar(im,ax=ax[0],orientation='horizontal')

im = ax[1].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.T[1,:,:],np.nan),shading='auto',cmap='cmo.thermal',vmin=-2,vmax=1)
plt.colorbar(im,ax=ax[1],orientation='horizontal')

im = ax[2].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.S[1,:,:],np.nan),shading='auto',cmap='cmo.haline')#,vmin=34,vmax=35)
plt.colorbar(im,ax=ax[2],orientation='horizontal')

#for q in [0,1,2]:
#    ax[q].set_xlim([-1.51e6,-1.49e6])
#    ax[q].set_ylim([-640000,-620000])

In [ ]:

fig,ax = plt.subplots(1,3,figsize=(20,10))
im = ax[0].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.drho,np.nan),shading='auto',cmap='cmo.amp',vmin=0,vmax=5e-4)
plt.colorbar(im,ax=ax[0],orientation='horizontal')

im = ax[1].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.Ta,np.nan),shading='auto',cmap='cmo.thermal')
plt.colorbar(im,ax=ax[1],orientation='horizontal')

im = ax[2].pcolormesh(layer.x,layer.y,np.where(layer.tmask==1,layer.Sa,np.nan),shading='auto',cmap='cmo.haline')
plt.colorbar(im,ax=ax[2],orientation='horizontal')

#for q in [0,1,2]:
#    ax[q].set_xlim([-1.51e6,-1.49e6])
#    ax[q].set_ylim([-640000,-620000])